In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from transformers import BertTokenizer

In [2]:
#data preprocessing
SEED = 42

In [3]:
file_directory = os.getcwd()

In [4]:
def preprocess(text):
    text = text.lower()
    if text[:4] == 'says':
        text = text[4:]
    elif text[:5] == 'print':
        text = text[5:]
    text = re.sub(r'-+', ' ', text)
    text = re.sub(r'_+', ' ', text)
    text = re.sub(r'(\n)+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    return text

In [5]:
def combine_dfs(df1, df2):
    return pd.concat([df1, df2])

In [6]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet
import random

In [7]:
def paraphrase(text):
    words = text.split()
    new_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            add = random.choice(synonyms[0].lemmas())
            new_words.append(add.name())
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [8]:
column_names = ['id', 'truth-value', 'statement', 'topics', 'speaker', 'speaker occupation', 'state', 'party', 'barely-true', 'false', 'half-true', 'true', 'POF', 'context']
df = pd.DataFrame()
current_df = pd.read_csv("./Liar dataset/train.tsv", sep="\t", names=column_names)


label_map = {
    'POF': 0,
    'false': 0,
    'barely-true': 0,
    'half-true': 1,
    'true': 1
}
label_columns = ['POF', 'false', 'barely-true', 'half-true', 'true']
current_df = current_df.dropna(subset=label_columns, how='all')
current_df = current_df[['statement', 'POF', 'false', 'barely-true', 'half-true', 'true']]
current_df['label'] = current_df[label_columns].idxmax(axis=1)
current_df['truth'] = current_df['label'].map(label_map)

"""weighted_sum = sum([current_data[col] * label_map[col] for col in label_columns])
total_counts = current_data[label_columns].sum(axis=1)
current_data['confidence'] = round((weighted_sum / total_counts), 2)"""
current_df = current_df[['statement', 'truth']]
current_df['statement'] = current_df['statement'].apply(preprocess)

In [9]:
current_df = current_df.sample(frac=1).reset_index(drop=True)

In [10]:
paraphrased_statements = pd.Series([paraphrase(statement) for statement in current_df[:len(current_df) // 2]['statement']])

In [11]:
paraphrased_statements = paraphrased_statements.apply(preprocess)
paraphrased_df = pd.DataFrame({'statement': paraphrased_statements, 'truth': current_df['truth'][:len(current_df) // 2]})

In [12]:
current_df = combine_dfs(paraphrased_df, current_df)

In [13]:
oversampler = RandomOverSampler(random_state=SEED)
X_resampled, y_resampled = oversampler.fit_resample(current_df[['statement']], current_df['truth'])
oversampled_df = pd.DataFrame({'statement': X_resampled['statement'], 'truth': y_resampled})

In [14]:
df = combine_dfs(df, oversampled_df)

In [15]:
current_df = pd.read_csv('./dataset 1/FakeNewsNet.csv')

current_df = current_df[['title', 'real']]
current_df = current_df.rename(columns={'title':'statement', 'real':'truth'})
current_df['truth'] = current_df['truth'].astype(int)
current_df['statement'] = current_df['statement'].apply(preprocess)

In [16]:
current_df = current_df.sample(frac=1).reset_index(drop=True)

In [17]:
paraphrased_statements = paraphrased_statements.apply(preprocess)
paraphrased_df = pd.DataFrame({'statement': paraphrased_statements, 'truth': current_df['truth'][:len(current_df) // 2]})

In [18]:
current_df = combine_dfs(paraphrased_df, current_df)

In [19]:
X_resampled, y_resampled = oversampler.fit_resample(current_df[['statement']], current_df['truth'])
oversampled_df = pd.DataFrame({'statement': X_resampled['statement'], 'truth': y_resampled})

In [20]:
df = combine_dfs(df, oversampled_df)

In [21]:
current_df = pd.read_csv('./KaggleFakeNews/train.csv')

In [22]:
current_df = current_df.dropna(how='all')
current_df = current_df.rename(columns={'text':'statement', 'label':'truth'})
current_df = current_df[['statement', 'truth']]
current_df['statement'] = current_df['statement'].astype(str).apply(preprocess)

In [23]:
current_df = current_df.sample(frac=1).reset_index(drop=True)

In [24]:
paraphrased_statements = pd.Series([paraphrase(statement) for statement in current_df['statement']])
paraphrased_df = pd.DataFrame({'statement': paraphrased_statements, 'truth': current_df['truth'][:len(current_df) // 2]})

In [25]:
current_df = combine_dfs(paraphrased_df, current_df)

In [26]:
current_df = current_df.dropna()

In [27]:
X_resampled, y_resampled = oversampler.fit_resample(current_df[['statement']], current_df['truth'])
oversampled_df = pd.DataFrame({'statement': X_resampled['statement'], 'truth': y_resampled})

In [28]:
df = combine_dfs(df, oversampled_df)

In [29]:
df['truth'].value_counts()

truth
1.0    49900
0.0    49900
Name: count, dtype: int64

In [30]:
from scipy.sparse import issparse, csr_matrix
from sklearn.preprocessing import MinMaxScaler

In [31]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.tree = self.grow_tree(X, y, 0)

    def grow_tree(self, X, y, depth):

        if depth >= self.max_depth or X.shape[0] < self.min_samples_split:
            return np.bincount(y).argmax()

        best_feature, best_threshold = self.find_best_split(X, y)
        if best_feature is None:
            return np.bincount(y).argmax()
        X_col = X[:, best_feature].toarray().flatten()    
        left_indices = X_col <= best_threshold
        right_indices = X_col > best_threshold

        if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
            # Return majority class if split is invalid
            return np.bincount(y).argmax()
        left = self.grow_tree(X[left_indices], y[left_indices], depth + 1)
        right = self.grow_tree(X[right_indices], y[right_indices], depth + 1)
        return {'feature': best_feature, 'threshold': best_threshold, 'left': left, 'right': right}

    def find_best_split(self, X, y):
        #iterate through every split and test gini
        n_features = X.shape[1]
        features = np.random.choice(n_features, int(np.sqrt(n_features)), replace=False)
        best_gini = 1.0
        best_feature, best_threshold = None, None
        for feature in features:
            X_col = X[:, feature].toarray().flatten()
            thresholds = np.unique(X_col[X_col > 0])
            for threshold in thresholds:
                left_indices = X_col <= threshold
                right_indices = X_col > threshold
                groups = [y[left_indices], y[right_indices]]

                gini = gini_impurity(groups, np.unique(y))

                if gini < best_gini:
                    best_gini = gini
                    best_threshold = threshold
                    best_feature = feature
            
        return best_feature, best_threshold

    def _predict_tree(self, X):
        predictions = []
        for row in X:
            node = self.tree
            while isinstance(node, dict):
                if row[node['feature']] <= node['threshold']:
                    node = node['left']
                else:
                    node = node['right']
            predictions.append(node)
        return predictions

In [32]:
def gini_impurity(groups, classes):
    n_instances = sum([len(group) for group in groups])
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            proportion = list(group).count(class_val) / size
            score += proportion ** 2
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [33]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_trees):
            X_sample, y_sample = random_sample(X, y)

            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, y)
            self.trees.append(tree)

    def predict(self, X):
        X_dense = X.toarray() if hasattr(X, "toarray") else X  # Handle sparse matrices
        predictions = np.array([tree._predict_tree(X_dense) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

In [34]:
def random_sample(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]

In [35]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [36]:
#model = RandomForestClassifier(n_estimators=100, random_state=42)
#model.fit(X_train, y_train)

In [37]:
#y_pred = model.predict(X_test)

In [38]:
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print(classification_report(y_test, y_pred))

In [39]:
#assume X parameter will be sparse
#print(X_train[0].indices)
#print(X_train[0].data)

In [40]:
#rf = RandomForest(n_trees=100, max_depth=10000)
#rf.fit(X_train, y_train)

#y_pred = rf.predict(X_test)

#accuracy = np.sum(y_pred == y_test) / len(y_test)
#print(f'accuracy: {accuracy}')

In [41]:
from fastai.text.all import *

In [42]:
from transformers import MobileBertTokenizer, TFAutoModelForSequenceClassification, AutoModelForSequenceClassification

In [43]:
df = df.sample(frac=1).reset_index(drop=True)

In [44]:
X_bert = df['statement'].astype(str)
y_bert = np.array(df['truth'])
X_bert_train, X_bert_test, y_bert_train, y_bert_test = train_test_split(X_bert, y_bert, test_size=.2, random_state=42)

In [45]:
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

In [46]:
X_bert_train = X_bert_train.tolist()
X_bert_test = X_bert_test.tolist()

In [47]:
train_tokenized = tokenizer(
    X_bert_train,
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors='pt'
)

test_tokenized = tokenizer(
    X_bert_test,
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors='pt'
)

In [54]:
train_input_ids = train_tokenized['input_ids']
train_attention_masks = train_tokenized['attention_mask']
train_labels = torch.tensor(y_bert_train, dtype=torch.long)

test_input_ids = test_tokenized['input_ids']
test_attention_mask = test_tokenized['attention_mask']
test_labels = torch.tensor(y_bert_test, dtype=torch.long)

In [56]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

In [57]:
import torch
from torch.utils.data import Dataset, DataLoader
train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=16)
class MyBertDataset(Dataset):
    def __init__(self, encodings, labels):
        '''
        encodings: a dict of Tensors (e.g., 'input_ids', 'attention_mask') from tokenizer(..., return_tensors='pt')
        labels: a list or tensor of labels (e.g., y_bert_train)
        '''
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        # Number of examples
        return len(self.labels)

    def __getitem__(self, idx):
        # For the given idx, gather all the tokenized inputs
        input_ids = self.encodings['input_ids'][idx].clone().detach()
        attention_mask = self.encodings['attention_mask'][idx].clone().detach()
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return input_ids, attention_mask, label
    def __testgetitem__(self, idx):
        # Suppose we load data from somewhere and convert it to tensors
        input_ids = torch.tensor([...])
        attention_mask = torch.tensor([...])
        label = torch.tensor(...)
        return input_ids, attention_mask, label

In [58]:
train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
train_attention_masks = torch.tensor(train_attention_masks, dtype=torch.long)
train_labels = torch.tensor(train_labels, dtype=torch.long)  # or torch.float for regression

test_input_ids = torch.tensor(test_input_ids, dtype=torch.long)
test_attention_mask = torch.tensor(test_attention_mask, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

C:\Users\kazuj\AppData\Local\Temp\ipykernel_35648\3737211109.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
C:\Users\kazuj\AppData\Local\Temp\ipykernel_35648\3737211109.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_attention_masks = torch.tensor(train_attention_masks, dtype=torch.long)
C:\Users\kazuj\AppData\Local\Temp\ipykernel_35648\3737211109.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels, dtype=torch.long)  

In [53]:
from transformers import MobileBertForSequenceClassification

class MyMobileBert(MobileBertForSequenceClassification):
    def forward(self, input_ids=None, attention_mask=None, labels=None, return_loss=False, **kwargs):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )
        if return_loss and labels is not None:
            return outputs.logits, outputs.loss
        # Return only the logits, which is what the fastai Learner will treat as predictions
        return outputs.logits


# Then instantiate:
model = MyMobileBert.from_pretrained("google/mobilebert-uncased", num_labels=2)

Some weights of MyMobileBert were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
train_ds = MyBertDataset(train_tokenized, y_bert_train)
test_ds  = MyBertDataset(test_tokenized,  y_bert_test)

In [59]:
def collate(batch):
    # Suppose each batch item is a tuple: (input_ids_list, attention_mask_list, label)
    input_ids = []
    attention_masks = []
    labels = []
    
    for b in batch:
        input_ids.append(b[0])
        attention_masks.append(b[1])
        labels.append(b[2])
        
    # Convert them all to tensors
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    attention_masks = torch.tensor(attention_masks, dtype=torch.long)
    labels = torch.tensor(labels, dtype=torch.long)
    
    return (input_ids, attention_masks, labels)

In [53]:
train_dl = DataLoader(train_ds, batch_size=2, collate_fn=collate)
test_dl = DataLoader(test_ds, batch_size=2, collate_fn=collate)

dls = DataLoaders(train_dl, test_dl)

In [60]:
learning_rate = .0002

In [61]:
from transformers import AdamW

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

C:\Users\kazuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [63]:
for epoch in range(3):
    model.train()
    for batch in train_dl:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_loss=True)
        loss = outputs[1]
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_dl:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            
            # Forward pass with validation loss computation
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_loss=True)
            logits = outputs[0]  # Logits from the model
            val_loss = F.cross_entropy(logits, labels)  # Compute loss for this batch
            
            # Accumulate total loss
            total_val_loss += val_loss.item()
            
            # Calculate predictions and accuracy
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    # Calculate average validation loss and accuracy
    avg_val_loss = total_val_loss / len(test_dl)  # Average loss per batch
    accuracy = correct / total  # Overall accuracy

    # Print metrics for this epoch
    print(f"Epoch {epoch + 1}:")
    print(f"  Average Validation Loss: {avg_val_loss:.4f}")
    print(f"  Validation Accuracy: {accuracy * 100:.2f}%")

Epoch 1:
  Average Validation Loss: 0.5592
  Validation Accuracy: 69.02%
Epoch 2:
  Average Validation Loss: 0.4729
  Validation Accuracy: 74.28%
Epoch 3:
  Average Validation Loss: 0.4190
  Validation Accuracy: 77.19%


In [84]:
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
}
torch.save(checkpoint, 'checkpoint.pth')

In [85]:
checkpoint = torch.load('checkpoint.pth', weights_only=True)

model = MyMobileBert.from_pretrained("google/mobilebert-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.train()

start_epoch = checkpoint['epoch']
loss = checkpoint['loss']

Some weights of MyMobileBert were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\kazuj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [108]:
model.to(device)
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)


In [109]:
model.eval()
total_val_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in test_dl:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        # Forward pass with validation loss computation
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_loss=True)
        logits = outputs[0]  # Logits from the model
        val_loss = F.cross_entropy(logits, labels)  # Compute loss for this batch
        
        # Accumulate total loss
        total_val_loss += val_loss.item()
        
        # Calculate predictions and accuracy
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

# Calculate average validation loss and accuracy
avg_val_loss = total_val_loss / len(test_dl)  # Average loss per batch
accuracy = correct / total  # Overall accuracy

# Print metrics for this epoch
print(f"Epoch {epoch + 1}:")
print(f"  Average Validation Loss: {avg_val_loss:.4f}")
print(f"  Validation Accuracy: {accuracy * 100:.2f}%")

Epoch 5:
  Average Validation Loss: 0.4283
  Validation Accuracy: 79.24%


In [74]:
print(start_epoch)

2


In [80]:
for param_group in optimizer.param_groups:
    param_group['lr'] = 1e-5  # Set a reasonable learning rate, e.g., 1e-5

In [82]:
for epoch in range(start_epoch, 5):
    model.train()
    total_train_loss = 0

    for batch in train_dl:
        optimizer.zero_grad()

        # Move data to device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs  # Raw logits from the model

        # Compute loss manually
        loss = torch.nn.functional.cross_entropy(logits, labels)

        # Backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Accumulate loss
        total_train_loss += loss.item()

    # Print average training loss
    avg_train_loss = total_train_loss / len(train_dl)
    print(f"Epoch {epoch + 1}: Average Training Loss: {avg_train_loss:.4f}")


Epoch 3: Average Training Loss: 0.3369
Epoch 4: Average Training Loss: 0.3240
Epoch 5: Average Training Loss: 0.3122


In [ ]:
for epoch in range(start_epoch, 5):
    model.train()
    for batch in train_dl:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_loss=True)
        loss = outputs[1]
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_dl:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            
            # Forward pass with validation loss computation
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, return_loss=True)
            logits = outputs[0]  # Logits from the model
            val_loss = F.cross_entropy(logits, labels)  # Compute loss for this batch
            
            # Accumulate total loss
            total_val_loss += val_loss.item()
            
            # Calculate predictions and accuracy
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    # Calculate average validation loss and accuracy
    avg_val_loss = total_val_loss / len(test_dl)  # Average loss per batch
    accuracy = correct / total  # Overall accuracy

    # Print metrics for this epoch
    print(f"Epoch {epoch + 1}:")
    print(f"  Average Validation Loss: {avg_val_loss:.4f}")
    print(f"  Validation Accuracy: {accuracy * 100:.2f}%")

Epoch 3:
  Average Validation Loss: 110.2467
  Validation Accuracy: 49.91%
Epoch 4:
  Average Validation Loss: nan
  Validation Accuracy: 49.91%
Epoch 5:
  Average Validation Loss: nan
  Validation Accuracy: 49.91%


In [107]:
checkpoint = torch.load('checkpoint.pth', weights_only=True)

model = MyMobileBert.from_pretrained("google/mobilebert-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.eval()

# Load the tokenizer
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

# Example headline
headline = "US Census found that 6 out of 10 americans are women"

# Preprocess the headline
tokens = tokenizer(
    headline,
    max_length=64,  # Ensure this matches the max length used during training
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

# Move to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokens = {key: val.to(device) for key, val in tokens.items()}

print('tokens input ids', tokens['input_ids'].shape)
print('tokens attention mask', tokens['attention_mask'].shape)
# Forward pass
with torch.no_grad():
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']


    # Forward pass with validation loss computation
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, return_loss=True)
    logits = outputs
    prediction = torch.argmax(logits, dim=1).item()  # Compute loss for this batch  # Get the predicted label

# Map prediction to label
labels = {0: "False", 1: "True"}  # Update this mapping based on your dataset
print(f"Prediction: {labels[prediction]}")

Some weights of MyMobileBert were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokens input ids torch.Size([1, 64])
tokens attention mask torch.Size([1, 64])
Prediction: True


In [ ]:
print('done')

In [ ]:
# Get one batch from your train_dataloader
"""batch = next(iter(train_dl))

# If your batch is something like (input_ids, attention_mask, labels):
input_ids, attention_mask, labels = batch

# Now check their types:
print(type(input_ids), type(attention_mask), type(labels))"""

In [60]:
from fastai.optimizer import Adam

In [ ]:
learn = Learner(
    dls,
    model,
    loss_func=CrossEntropyLossFlat(),
    metrics=accuracy,
)

# 9) Use lr_find()
lr = learn.lr_find()
print("lr_find results:", lr)

In [ ]:
print('done')

In [ ]:
torch.save(model.state_dict(), "my_model_weights.pt")

In [ ]:
# Recreate the same model architecture
model = MyMobileBert.from_pretrained("google/mobilebert-uncased", num_labels=2)  # same class & config used before

# Load state dict
model.load_state_dict(torch.load("my_model_weights.pt"))
model.eval()  # or model.train() if you're continuing training

In [ ]:
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.mixed_precision import Policy, set_global_policy

# Set the global policy for mixed precision
policy = Policy('mixed_float16')
set_global_policy(policy)

print("Mixed precision policy set:", policy)

In [ ]:
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)

In [ ]:
model.save_pretrained('./new_model')
tokenizer.save_pretrained('./new_model')

In [ ]:
test_data = pd.read_csv("./Liar dataset/test.tsv", sep="\t", names=column_names)

label_map = {
    'POF': 0,
    'false': 0,
    'barely-true': 0,
    'half-true': 1,
    'true': 1
}
label_columns = ['POF', 'false', 'barely-true', 'half-true', 'true']

test_data = test_data.dropna(subset=label_columns, how='all')
test_data = test_data[['statement', 'POF', 'false', 'barely-true', 'half-true', 'true']]
test_data['label'] = test_data[label_columns].idxmax(axis=1)
test_data['truth'] = test_data['label'].map(label_map)

"""weighted_sum = sum([current_data[col] * label_map[col] for col in label_columns])
total_counts = current_data[label_columns].sum(axis=1)
current_data['confidence'] = round((weighted_sum / total_counts), 2)"""
test_data = test_data[['statement', 'truth']]
test_data['statement'] = test_data['statement'].apply(preprocess)

In [ ]:
feats = test_data['statement'].astype(str).tolist()
labels = test_data['truth']

tst_tokenized = tokenizer(
    feats,
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors='tf'
)
tst_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': tst_tokenized['input_ids'], 'attention_mask': tst_tokenized['attention_mask']},
    labels
)).batch(16)

In [ ]:
from tqdm import tqdm

batch_size = len(test_tokenized['input_ids']) // 32  # Adjust based on your hardware
outputs = []

for i in tqdm(range(0, len(test_tokenized['input_ids']), batch_size)):
    batch_input_ids = test_tokenized['input_ids'][i:i+batch_size]
    batch_attention_mask = test_tokenized['attention_mask'][i:i+batch_size]
    batch_output = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
    outputs.append(batch_output.logits)

In [ ]:
all_logits = tf.concat(outputs, axis=0)

print(all_logits.shape)

In [ ]:
predicted_classes = tf.argmax(all_logits, axis=-1)
correct_predictions = tf.reduce_sum(tf.cast(predicted_classes == y_bert_test, tf.float32))
accuracy = correct_predictions / len(y_bert_test)

print(f"Model Accuracy: {accuracy.numpy():.2%}")

In [ ]:
"""current_data_1 = pd.read_csv('./dataset 2/dataset/gossipcop_fake.csv')
current_data_1 = current_data_1.dropna(how='all')
current_data_1 = current_data_1[['title']]
current_data_1 = current_data_1.rename(columns={'title':'statement'})
current_data_1['truth'] = 0"""

In [ ]:
"""current_data_2 = pd.read_csv('./dataset 2/dataset/gossipcop_real.csv')
current_data_2 = current_data_2.dropna(how='all')
current_data_2 = current_data_2[['title']]
current_data_2 = current_data_2.rename(columns={'title':'statement'})
current_data_2['truth'] = 1"""

In [ ]:
"""current_data = pd.concat([current_data_1, current_data_2])
X = current_data['statement']
y = current_data['truth']

vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_vectorized, y)

resampled_df = pd.DataFrame({'statement': vectorizer.inverse_transform(X_resampled),  # Attempt to reverse transform
                             'truth': y_resampled})
current_data = resampled_df"""

In [ ]:
#data = pd.concat([data, current_data])

In [ ]:
"""current_df_1 = pd.read_csv('./dataset 2/dataset/politifact_fake.csv')
current_df_1 = current_df_1.dropna(how='all')
current_df_1 = current_df_1[['title']]
current_df_1 = current_df_1.rename(columns={'title':'statement'})
current_df_1['truth'] = 0
current_df_1['statement'] = current_df_1['statement'].apply(preprocess)"""

In [ ]:
"""current_df_2 = pd.read_csv('./dataset 2/dataset/politifact_real.csv')
current_df_2 = current_df_2.dropna(how='all')
current_df_2 = current_df_2[['title']]
current_df_2 = current_df_2.rename(columns={'title':'statement'})
current_df_2['truth'] = 1
current_df_2['statement'] = current_df_2['statement'].apply(preprocess)"""

In [ ]:
#current_df = pd.concat([current_df_1, current_df_2])
"""X = current_data['statement']
y = current_data['truth']

vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_vectorized, y)

resampled_df = pd.DataFrame({'statement': vectorizer.inverse_transform(X_resampled),  # Attempt to reverse transform
                             'truth': y_resampled})
current_data = resampled_df"""

In [ ]:
#logistic regression model from scratch

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
class LogisticRegression:
    def __init__(self):
        ...
        
    def sigmoid(self, n):
        return 1 / (1 + np.exp(-n))
        
    def initialize_weights(self, n_features):
        weights = np.zeros(n_features)
        bias = 0
        return weights, bias
        
    def predict(self, X, weights, bias):
        linear_model = X.dot(weights) + bias
        predictions = self.sigmoid(linear_model)
        return predictions
        
    def calculate_loss(self, y_true, y_pred):
        n = len(y_true)
        loss = (-1/n) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss
    
    def gradient_descent(self, X, y, weights, bias, lr):
        n = X.shape[0]
    
        y_pred = self.predict(X, weights, bias)
    
        dw = X.T.dot(y_pred - y) / n
        db = np.sum(y_pred - y) / n
    
        weights -= lr * dw
        bias -= lr * db
    
        return weights, bias

    def train(self, X, y, lr=.1, epochs=1000, batch_size=500):
        n_features = X.shape[1]
    
        weights, bias = self.initialize_weights(n_features)
    
        losses = []
    
        for epoch in range(epochs):
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                
                
                weights, bias = self.gradient_descent(X_batch, y_batch, weights, bias, lr)
    
            y_pred = self.predict(X, weights, bias)
            loss = self.calculate_loss(y, y_pred)
            losses.append(loss)
    
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")
        return weights, bias, losses

    def classify(self, X, weights, bias, threshold=.5):
        probabilities = self.predict(X, weights, bias)
        return [1 if p >= threshold else 0 for p in probabilities]

In [ ]:
#USE FOR THE SKLEARN MODEL
#vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

#X = data['statement'].astype(str)

In [ ]:
#USE FOR THE SKLEARN MODEL
"""X_tfidf = vectorizer.fit_transform(X)
y = np.array(data['truth'])"""

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=.2, random_state=42) #USE FOR THE SKLEARN MODEL

In [ ]:
#DON'T USE FOR SKLEARN
#regressor = LogisticRegression()
#weights, bias, losses = regressor.train(X_train, y_train)
#y_pred = regressor.classify(X_test, weights, bias)
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print(classification_report(y_test, y_pred))

In [ ]:
#logistic regression using sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
#model = LogisticRegression(max_iter=1000, solver='lbfgs')
#model.fit(X_train, y_train)

In [ ]:
#y_pred = model.predict(X_test)
#y_prob = model.predict_proba(X_test)[:, 1]

In [ ]:
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print(classification_report(y_test, y_pred))

In [ ]:
#random forest